# Feed-Forward Neural Network

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from itertools import product
from pathlib import Path
from tqdm.auto import trange
import pickle

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

from system_identification.ffnn import FeedForwardNeuralNetwork
from system_identification.load_assignment_data import load_net_example_ff

## 1D

In [3]:
def foo(x):
    return 0.8 * np.tanh(x * 0.4 - 0.7) + 2.1

# def f(x):
#     return x[0]**2

# def f(x):
#     return -0.8 * np.tanh(x[0] * 3) + x[1]**2 + 1

inputs = np.linspace(-5, 10, 1000)
reference_outputs = np.array(list(map(foo, inputs)))
reference_outputs_noisy = reference_outputs + (np.random.random(reference_outputs.shape) - 0.5) * 0.1

inputs = inputs.reshape(-1, 1, 1)
reference_outputs = reference_outputs.reshape(-1, 1, 1)
reference_outputs_noisy = reference_outputs_noisy.reshape(-1, 1, 1)

In [4]:
plt.figure()
plt.plot(inputs[:, 0, 0], reference_outputs[:, 0, 0], "o", markersize=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Levenberg-Marquardt training

In [6]:
model = FeedForwardNeuralNetwork.new(
    n_inputs=1,
    n_outputs=1,
    n_hidden=3,
    range=[[-1, 1]],
    log_dir="./ffnn_exp",
)
model

<FeedForwardNeuralNetwork
 n_inputs=1
 n_hidden=3
 n_outputs=1>

In [7]:
model.train(
    inputs,
    reference_outputs,
    epochs=10000,
    goal=1e-6,
    train_log_freq=1,
    method="trainlm",
    mu=10.,
    alpha=0.95)

  0%|          | 0/10000 [00:00<?, ?it/s]

Goal met


In [8]:
fig = plt.figure()
outputs = model.evaluate(inputs)
plt.plot(inputs.squeeze(), reference_outputs.squeeze())
plt.plot(inputs.squeeze(), reference_outputs_noisy.squeeze(), ".", markersize=1)
plt.plot(inputs.squeeze(), outputs.squeeze())

plt.figure()
model.training_log.error.plot()
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Back propagation

In [9]:
model = FeedForwardNeuralNetwork.new(
    n_inputs=1,
    n_outputs=1,
    n_hidden=1,
    range=[[-1, 1]],
    log_dir="./ffnn_exp",
)
model

<FeedForwardNeuralNetwork
 n_inputs=1
 n_hidden=1
 n_outputs=1>

In [10]:
model.train(
    inputs,
    reference_outputs,
    epochs=10000,
    goal=1e-6,
    train_log_freq=1,
    method="trainbp",
    eta=0.001,
    alpha=1.001)

  0%|          | 0/10000 [00:00<?, ?it/s]

Goal met


In [11]:
fig = plt.figure()
outputs = model.evaluate(inputs)
plt.plot(inputs.squeeze(), reference_outputs.squeeze())
plt.plot(inputs.squeeze(), reference_outputs_noisy.squeeze(), ".", markersize=1)
plt.plot(inputs.squeeze(), outputs.squeeze())

plt.figure()
model.training_log.error.plot()
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D

In [26]:
input_range = np.array([(-1, 1), (-3, 5)])


def foo(x):
    return -x[0]**2 + np.sin(x[1]*2)


resolution = 100
inputs = np.array(tuple(product(np.linspace(*input_range[0, :], resolution),
                                np.linspace(*input_range[1, :], resolution))))

reference_outputs = np.array(list(map(foo, inputs)))
reference_outputs_noisy = reference_outputs + (np.random.random(reference_outputs.shape) - 0.5) * 0.1

inputs = inputs[..., None]
reference_outputs = reference_outputs.reshape(-1, 1, 1)
reference_outputs_noisy = reference_outputs_noisy.reshape(-1, 1, 1)

In [29]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Levenberg-Marquardt training

In [41]:
model = FeedForwardNeuralNetwork.new(
    n_inputs=2,
    n_outputs=1,
    n_hidden=10,
    range=input_range,
    log_dir="./ffnn_exp",
)
model

<FeedForwardNeuralNetwork
 n_inputs=2
 n_hidden=10
 n_outputs=1>

In [42]:
model.train(
    inputs,
    reference_outputs,
    epochs=1000,
    goal=1e-6,
    train_log_freq=1,
    method="trainlm",
    mu=10.,
    alpha=0.99
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [43]:
fig = plt.figure()
output = model.evaluate(inputs)
ax = fig.add_subplot(projection='3d')
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], output.squeeze(), s=0.1)

plt.figure()
model.training_log.error.plot()
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Back propagation training

In [59]:
model = FeedForwardNeuralNetwork.new(
    n_inputs=2,
    n_outputs=1,
    n_hidden=10,
    range=input_range,
    log_dir="./ffnn_exp",
)
model

<FeedForwardNeuralNetwork
 n_inputs=2
 n_hidden=10
 n_outputs=1>

In [60]:
model.train(
    inputs,
    reference_outputs,
    epochs=10000,
    goal=1e-6,
    train_log_freq=1,
    method="trainbp",
    eta=.01,
    alpha=1.001
)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [61]:
fig = plt.figure()
output = model.evaluate(inputs)
ax = fig.add_subplot(projection='3d')
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], output.squeeze(), s=0.1)

plt.figure()
model.training_log.error.plot()
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Example

In [144]:
data_dir_path = Path().cwd().parent.parent / "assignment"
example_nn = load_net_example_ff(data_dir_path)
example_nn

In [73]:
x = np.array([[1, 2, 3]]).T
y = np.array([[4, 5, 6, 7]])

x @ y

array([[ 4,  5,  6,  7],
       [ 8, 10, 12, 14],
       [12, 15, 18, 21]])

In [85]:
np.zeros(3)

array([0., 0., 0.])

In [86]:
import sympy as sp

In [120]:
x = sp.Matrix(sp.symbols("x_{00} x_{01} x_{02} x_{10} x_{11} x_{12}")).reshape(2, 3)
y = sp.Matrix(sp.symbols("y_0:3")).T

In [121]:
x

Matrix([
[x_{00}, x_{01}, x_{02}],
[x_{10}, x_{11}, x_{12}]])

In [124]:
y

Matrix([[y_0, y_1, y_2]])

In [129]:
qwerty = sp.zeros(3, 1)
for j, k in product(range(3), range(2)):
    qwerty[j] += x[k, j] * y[j]
qwerty

Matrix([
[x_{00}*y_0 + x_{10}*y_0],
[x_{01}*y_1 + x_{11}*y_1],
[x_{02}*y_2 + x_{12}*y_2]])

ShapeError: Matrix size mismatch: (3, 2) * (1, 3).